In [1]:
import pandas as pd
import numpy as np
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1

# Get reusable credentials
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

# Create "Client" objects
client = bigquery.Client(credentials=credentials, project=your_project_id)
storage_client = bigquery_storage_v1beta1.BigQueryStorageClient(credentials=credentials)

# # Construct a reference to the "chicago_crime" dataset
# dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# # API request - fetch the dataset
# dataset = client.get_dataset(dataset_ref)

# # API request - fetch the table
# table_ref = dataset.table('crime')
# table = client.get_table(table_ref)

In [2]:
table = bigquery_storage_v1beta1.types.TableReference()
table.project_id = "bigquery-public-data"
table.dataset_id = "chicago_crime"
table.table_id = "crime"

read_options = bigquery_storage_v1beta1.types.TableReadOptions()
# read_options.selected_fields.append("species_common_name")
# read_options.selected_fields.append("fall_color")

session = storage_client.create_read_session(
    table,
    parent="projects/{}".format(your_project_id),
    read_options=read_options,
    format_=bigquery_storage_v1beta1.enums.DataFormat.ARROW,
    sharding_strategy=(
        bigquery_storage_v1beta1.enums.ShardingStrategy.LIQUID
    ),
)

stream = session.streams[0]
position = bigquery_storage_v1beta1.types.StreamPosition(stream=stream)
reader = storage_client.read_rows(position)
dataframe = reader.to_dataframe(session)
print(dataframe.head())

   unique_key case_number                      date                  block  \
0    11157208    JA522195 2013-11-22 20:30:00+00:00       095XX S AVENUE M   
1    10647365    HZ398263 2015-07-16 00:00:00+00:00       132XX S AVENUE N   
2     4927040    HM538120 2002-06-01 00:01:00+00:00  082XX S RIDGELAND AVE   
3     9088700    HW233645 2003-01-01 01:00:00+00:00     112XX S ALBANY AVE   
4    11509297    JB518435 2015-11-16 00:01:00+00:00     053XX S HAMLIN AVE   

   iucr                primary_type                          description  \
0  2826               OTHER OFFENSE       HARASSMENT BY ELECTRONIC MEANS   
1  1153          DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT OVER $ 300   
2  1562                 SEX OFFENSE            AGG CRIMINAL SEXUAL ABUSE   
3  0266         CRIM SEXUAL ASSAULT                            PREDATORY   
4  1753  OFFENSE INVOLVING CHILDREN        SEX ASSLT OF CHILD BY FAM MBR   

  location_description  arrest  domestic  ...  ward  community_area  fbi_c

In [6]:
dataframe.shape

(7018864, 22)